In [132]:
import sqlite3
import random
import numpy as np
from faker import Faker
import pandas as pd
from datetime import date, timedelta

# Inicializar o Faker para dados em Português
faker = Faker('pt_PT')

In [133]:
Departamentos = ["Recursos Humanos", "Tecnologia da Informação", "Financeiro", "Marketing", "Vendas", "Qualidade", "Atendimento ao Cliente", "Jurídico"]
Cargos = ["Gerente de Projetos", "Analista de Dados", "Desenvolvedor de Software Sénior", "Desenvolvedor de Software Júnior", "Engenheiro de DevOps","Designer UI/UX","Analista de Sistemas","Técnico de Suporte de TI","Analista de Recursos Humanos", "Especialista em Recrutamento", "Gerente de Marketing", "Especialista em Marketing Digital", "Analista de Mídia Social","Redator (Copywriter)","Gerente de Vendas","Executivo de Contas","Representante de Vendas Interno","Analista Financeiro","Contador","Gerente Financeiro","Analista de Qualidade","Inspetor de Qualidade","Agente de Atendimento ao Cliente","Supervisor de Atendimento ao Cliente",  "Especialista em Sucesso do Cliente",  "Advogado Corporativo","Assistente Jurídico","Assistente Administrativo","Recepcionista","Diretor de Operações (COO)"]

In [134]:
def criar_departamentos():
    lista_departamentos = []
    for i in range(1, len(Departamentos) + 1):
        lista_departamentos.append({
            "ID_depart": i,
            "Nome": Departamentos[i-1],
            "ID_gerente": None # Começa como Nulo
        })
    return pd.DataFrame(lista_departamentos)



In [135]:
def criar_funcionarios(n=50):
    lista_funcionarios = []
    for i in range(1, n + 1):
        lista_funcionarios.append({
            "ID_fun": i,
            "NIF": str(283684274 + i),
            "Primeiro_nome": faker.first_name(),
            "Ultimo_nome": faker.last_name(),
            "Nome_rua": faker.street_name(),
            "Nome_localidade": faker.city(),
            "Codigo_postal": faker.postcode(),
            "Num_telemovel": faker.phone_number(),
            "Email": faker.unique.email(),
            "Data_nascimento": faker.date_of_birth(minimum_age=18, maximum_age=65),
            "Cargo": random.choice([cargo for cargo in Cargos if cargo != "Gerente"]),
            "ID_depart": np.random.randint(1, len(Departamentos) + 1)
        })
    return pd.DataFrame(lista_funcionarios)

In [136]:
Departamentos = criar_departamentos()
Funcionarios = criar_funcionarios()
Departamentos["ID_gerente"] = random.sample(list(Funcionarios["ID_fun"]), len(Departamentos))



In [137]:
Departamentos


,ID_depart,Nome,ID_gerente
0,1,Recursos Humanos,27
1,2,Tecnologia da Informação,28
2,3,Financeiro,41
3,4,Marketing,42
4,5,Vendas,46
5,6,Qualidade,37
6,7,Atendimento ao Cliente,21
7,8,Jurídico,20


In [138]:
Funcionarios = criar_funcionarios(100)
Funcionarios


,ID_fun,NIF,Primeiro_nome,Ultimo_nome,Nome_rua,Nome_localidade,Codigo_postal,Num_telemovel,Email,Data_nascimento,Cargo,ID_depart
0,1,283684275,Yara,Gomes,Avenida João Monteiro,Castelo Branco,5194-326,(351) 272005944,caetanacunha@example.net,1976-02-14,Supervisor de Atendimento ao Cliente,7
1,2,283684276,Francisca,Batista,Rua de Oliveira,Fátima,7640-732,965 867 104,zcarneiro@example.org,1973-07-11,Redator (Copywriter),7
2,3,283684277,Alexandra,Assunção,Largo Nunes,Lamego,2177-801,(351) 922 753 756,calves@example.org,1988-08-30,Analista de Dados,3
3,4,283684278,Filipa,Leite,Praça Lisandro Miranda,Tomar,5828-504,(351) 919 963 200,julia04@example.com,1985-05-23,Especialista em Recrutamento,6
4,5,283684279,Fábio,Campos,Avenida de S. Lázaro,Covilhã,8040-415,(351) 960588308,brian79@example.org,2001-09-02,Analista Financeiro,7
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,283684370,Diogo,Antunes,Largo Igor Vaz,Covilhã,4532-428,960 468 175,gasparcardoso@example.org,1978-06-14,Executivo de Contas,8
96,97,283684371,Gabriela,Andrade,Av de Freitas,Aveiro,6334-684,(351) 915 097 723,costamartim@example.com,1977-07-09,Desenvolvedor de Software Júnior,3
97,98,283684372,Manuel,Maia,Praça Costa,Loures,6158-105,(351) 962 720 586,sabryan@example.net,1974-06-17,Analista de Dados,6
98,99,283684373,Maria,Ribeiro,Avenida Guerreiro,Lagos,9364-310,(351) 932439047,netolourenco@example.net,1982-03-29,Analista Financeiro,1


In [139]:
def criar_remuneracoes_salarios_beneficios(funcionarios_df):
    """
    Gera um HISTÓRICO de dados para Remuneracoes, Salario e Beneficios.
    Garante que nenhuma Data_fim ultrapassa 2025-12-31.
    """
    MAX_DATE = date(2025, 12, 31)
    lista_remuneracoes = []
    lista_salarios = []
    lista_beneficios = []

    tipos_de_beneficios = ['Subsídio Alimentação', 'Seguro Saúde', 'Carro Empresa', 'Subsídio Transporte', 'Telemóvel Empresa']

    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        num_registos_historicos = random.randint(1, 4)
        
        salario_atual = round(np.random.normal(loc=1300, scale=400), 2)
        if salario_atual < 950: salario_atual = 950
            
        # Garantir que o primeiro contrato começa, no máximo, no final de 2024
        max_start_date = date(2024, 12, 31)
        data_inicio_atual = faker.date_between(start_date='-5y', end_date=max_start_date)

        for i in range(num_registos_historicos):
            
            # Se a data de início já passou de 2025, não criamos mais registos
            if data_inicio_atual > MAX_DATE:
                break
                
            data_inicio_registo = data_inicio_atual
            proxima_data_inicio = None
            
            if i < num_registos_historicos - 1:
                # Se NÃO for o último registo (histórico)
                proxima_data_inicio = data_inicio_registo + timedelta(days=random.randint(350, 400))
                data_fim_registo = proxima_data_inicio - timedelta(days=1)
                
                # --- APLICA A REGRA DE DATA MÁXIMA ---
                if data_fim_registo > MAX_DATE:
                    data_fim_registo = MAX_DATE
                    proxima_data_inicio = MAX_DATE + timedelta(days=1) # Força o próximo a parar
            
            else:
                # Se FOR o último registo (o atual)
                if random.random() < 0.5:
                    # Tem uma data de fim. Calcular e limitar a 2 anos (para não ir muito longe)
                    data_fim_calculada = data_inicio_registo + timedelta(days=random.randint(90, 730))
                    
                    # --- APLICA A REGRA DE DATA MÁXIMA ---
                    if data_fim_calculada > MAX_DATE:
                        data_fim_registo = MAX_DATE
                    else:
                        data_fim_registo = data_fim_calculada
                else:
                    data_fim_registo = None # 50% de chance de ser Nulo
            
            # --- 1. Adicionar registo de Remuneração ---
            remuneracao_record = {
                "ID_fun": id_fun,
                "Data_inicio": data_inicio_registo,
                "Data_fim": data_fim_registo
            }
            lista_remuneracoes.append(remuneracao_record)
            
            # --- 2. Adicionar registo de Salário (versão simplificada) ---
            salario_record = {
                "ID_fun": id_fun,
                "Data_inicio": data_inicio_registo,
                "salario_bruto": round(salario_atual, 2)
            }
            lista_salarios.append(salario_record)
            
            # --- 3. Adicionar Benefícios (versão simplificada) ---
            if random.random() < 0.8:
                num_beneficios = random.randint(1, 3)
                beneficios_para_este_fun = random.sample(tipos_de_beneficios, num_beneficios)
                
                for tipo_beneficio in beneficios_para_este_fun:
                    beneficio_record = {
                        "ID_fun": id_fun,
                        "Data_inicio": data_inicio_registo,
                        "tipo": tipo_beneficio,
                        "valor": round(random.uniform(30, 450), 2)
                    }
                    lista_beneficios.append(beneficio_record)
            
            # --- 4. Preparar a próxima iteração ---
            salario_atual *= random.uniform(1.02, 1.10) # Aumento
            if proxima_data_inicio:
                data_inicio_atual = proxima_data_inicio

    remuneracoes_df = pd.DataFrame(lista_remuneracoes)
    salarios_df = pd.DataFrame(lista_salarios)
    beneficios_df = pd.DataFrame(lista_beneficios)
    
    return remuneracoes_df, salarios_df, beneficios_df

In [140]:
Remuneracoes, Salarios, Beneficios = criar_remuneracoes_salarios_beneficios(Funcionarios)

In [141]:
Remuneracoes

,ID_fun,Data_inicio,Data_fim
0,1,2023-01-22,2024-01-13
1,1,2024-01-14,None
2,2,2023-07-15,2024-07-25
3,3,2021-05-08,2022-05-02
4,4,2024-10-26,2025-12-31
...,...,...,...
223,98,2023-06-24,None
224,99,2024-11-24,None
225,100,2020-12-03,2021-12-14
226,100,2021-12-15,2023-01-10


In [142]:
Salarios

,ID_fun,Data_inicio,salario_bruto
0,1,2023-01-22,950.00
1,1,2024-01-14,970.18
2,2,2023-07-15,1644.84
3,3,2021-05-08,1158.88
4,4,2024-10-26,1380.98
...,...,...,...
223,98,2023-06-24,1286.97
224,99,2024-11-24,1570.24
225,100,2020-12-03,1226.31
226,100,2021-12-15,1335.03


In [143]:
Beneficios

,ID_fun,Data_inicio,tipo,valor
0,1,2023-01-22,Carro Empresa,352.95
1,1,2024-01-14,Seguro Saúde,297.49
2,2,2023-07-15,Carro Empresa,132.43
3,3,2021-05-08,Seguro Saúde,211.62
4,6,2021-04-18,Subsídio Transporte,295.49
...,...,...,...,...
357,98,2023-06-24,Telemóvel Empresa,206.59
358,98,2023-06-24,Subsídio Alimentação,397.40
359,98,2023-06-24,Subsídio Transporte,311.85
360,99,2024-11-24,Carro Empresa,326.43


In [144]:
def criar_ferias(funcionarios_df):
    """
    NÃO TEM O NUM_DIAS_PERMITIDOS
    Gera um histórico de férias para os funcionários.
    Cada funcionário pode ter entre 0 a 4 registos de férias.
    """
    lista_ferias = []
    estados_possiveis = ['Aprovado', 'Aprovado', 'Aprovado', 'Aprovado', 'Rejeitado', 'Por aprovar']
    
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        num_periodos_ferias = random.randint(0, 4)
        
        for _ in range(num_periodos_ferias):
            data_inicio = faker.date_between(start_date='-3y', end_date='today')
            num_dias = random.randint(2, 15)
            data_fim = data_inicio + timedelta(days=num_dias)
            
            ferias_record = {
                "ID_fun": id_fun,
                "data_inicio": data_inicio,
                "data_fim": data_fim,
                "num_dias": num_dias,
                "estado_aprov": random.choice(estados_possiveis)
            }
            lista_ferias.append(ferias_record)
            
    return pd.DataFrame(lista_ferias)

In [145]:
Ferias = criar_ferias(Funcionarios)
Ferias

,ID_fun,data_inicio,data_fim,num_dias,estado_aprov
0,1,2024-03-22,2024-03-25,3,Aprovado
1,1,2024-05-23,2024-05-31,8,Por aprovar
2,2,2023-09-28,2023-10-12,14,Rejeitado
3,2,2024-09-21,2024-09-29,8,Por aprovar
4,4,2022-11-27,2022-12-04,7,Aprovado
...,...,...,...,...,...
164,97,2023-06-22,2023-07-07,15,Por aprovar
165,98,2023-11-12,2023-11-19,7,Aprovado
166,99,2023-02-20,2023-03-06,14,Por aprovar
167,99,2023-03-17,2023-03-20,3,Aprovado


In [146]:
def criar_dependentes(funcionarios_df):
    """
    Gera uma lista de dependentes e depois limpa os dados para garantir que
    nenhum funcionário tem mais do que um dependente com o parentesco 'Pai/Mãe'.

    Regras adicionadas:
    - 'Pai/Mãe' tem sempre data de nascimento ANTERIOR à do funcionário (>= 18 e <= 50 anos mais velho, limitado a 100 anos de idade total)
    - 'Filho(a)' tem sempre data de nascimento POSTERIOR à do funcionário (pelo menos 1 dia depois)
    """
    lista_dependentes = []
    # Usamos 'Pai/Mãe' como uma única categoria para simplificar
    parentescos_possiveis = ['Filho(a)', 'Cônjuge', 'Enteado(a)', 'Pai/Mãe', 'Filho(a)']  # Repetimos 'Filho(a)' para aumentar a probabilidade
    sexos_possiveis = ['Masculino', 'Feminino', 'Outro']

    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        data_nasc_funcionario = funcionario['Data_nascimento']

        if random.random() < 0.7:
            num_dependentes = random.randint(1, 4)  # Aumentei para 4 para ter mais chance de criar duplicados para limpar

            for _ in range(num_dependentes):
                parentesco = random.choice(parentescos_possiveis)

                # Lógica para gerar data de nascimento conforme o parentesco
                if parentesco == 'Pai/Mãe':
                    # Calcular idade atual do funcionário
                    idade_funcionario = (date.today() - data_nasc_funcionario).days // 365
                    idade_minima_pai = idade_funcionario + 18
                    idade_maxima_pai = min(idade_funcionario + 50, 100)  # Limitar a 100 anos de idade
                    # Garantir limites válidos (se o funcionário for muito jovem, ajusta)
                    if idade_minima_pai > idade_maxima_pai:
                        idade_minima_pai = idade_maxima_pai - 1 if idade_maxima_pai > 0 else 18
                    data_nascimento = faker.date_of_birth(minimum_age=idade_minima_pai, maximum_age=idade_maxima_pai)
                elif parentesco == 'Filho(a)':
                    # Filho deve ser mais novo: gerar data depois da do funcionário
                    start_child = data_nasc_funcionario + timedelta(days=1)
                    # Se por algum motivo o funcionário nasceu "hoje", o filho terá a mesma regra +1 dia
                    if start_child >= date.today():
                        data_nascimento = start_child
                    else:
                        # Usa API explícita para limites de datas
                        data_nascimento = faker.date_between_dates(date_start=start_child, date_end=date.today())
                else:
                    # Outros dependentes podem ter qualquer idade plausível (0 a 80 anos de idade agora)
                    data_nascimento = faker.date_of_birth(minimum_age=0, maximum_age=80)

                dependente_record = {
                    "ID_Fun": id_fun,
                    "nome": faker.name(),
                    "sexo": random.choice(sexos_possiveis),
                    "data_nascimento": data_nascimento,
                    "parentesco": parentesco
                }
                lista_dependentes.append(dependente_record)

    # --- PASSO DE LIMPEZA ---

    # 1. Criar o DataFrame com os dados
    if not lista_dependentes:
        return pd.DataFrame()  # Retorna um DataFrame vazio se nenhum dependente foi gerado

    dependentes_bruto_df = pd.DataFrame(lista_dependentes)

    # 2. Separar os pais/mães dos outros dependentes
    pais_maes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] == 'Pai/Mãe']
    outros_dependentes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] != 'Pai/Mãe']

    # 3. Remover duplicados APENAS da parte dos pais/mães
    pais_maes_unicos_df = pais_maes_df.drop_duplicates(subset=['ID_Fun', 'parentesco'], keep='first')

    # 4. Juntar novamente as duas partes para ter o DataFrame final e limpo
    dependentes_final_df = pd.concat([pais_maes_unicos_df, outros_dependentes_df], ignore_index=True)

    return dependentes_final_df

In [147]:
Dependentes = criar_dependentes(Funcionarios)
Dependentes

,ID_Fun,nome,sexo,data_nascimento,parentesco
0,3,João Carneiro,Feminino,1945-01-09,Pai/Mãe
1,5,Manuel Barros,Feminino,1966-08-20,Pai/Mãe
2,9,Ângelo Leite,Feminino,1949-05-23,Pai/Mãe
3,12,Lucas Campos,Feminino,1942-03-08,Pai/Mãe
4,13,Rodrigo da Tavares,Masculino,1937-07-14,Pai/Mãe
...,...,...,...,...,...
154,97,Cláudio Anjos,Feminino,1989-08-05,Filho(a)
155,97,Andreia Neto,Masculino,1981-06-12,Filho(a)
156,98,Benjamim Macedo,Masculino,1950-04-18,Enteado(a)
157,99,Carminho Faria,Outro,1998-10-16,Cônjuge


In [148]:
def criar_faltas(funcionarios_df):
    """
    Gera um histórico de faltas para os funcionários.
    Cada funcionário terá entre 0 e 10 faltas registadas.
    """
    lista_faltas = []
    justificacoes_comuns = ["Consulta médica","Doença súbita","Apoio à família","Assuntos pessoais urgentes","Procedimentos legais/burocráticos","Avaria de transporte", None, None] # Para simular faltas injustificadas

    # Itera sobre cada funcionário para criar o seu histórico de faltas
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # Cada funcionário terá um número aleatório de faltas (de 0 a 10)
        num_faltas = int(np.random.normal(loc=3, scale=2))
        
        # Para evitar que as faltas do mesmo funcionário sejam todas no mesmo dia,
        # geramos um conjunto de datas únicas para este funcionário
        datas_possiveis = [faker.date_between(start_date='-3y', end_date='today') for _ in range(num_faltas)]
        datas_unicas = set(datas_possiveis) # 'set' remove datas duplicadas automaticamente
        
        for data_falta in datas_unicas:
            falta_record = {
                "ID_Fun": id_fun,
                "Data": data_falta,
                "Justificacao": random.choice(justificacoes_comuns)
            }
            lista_faltas.append(falta_record)
            
    return pd.DataFrame(lista_faltas)

In [149]:
Faltas = criar_faltas(Funcionarios)
Faltas

,ID_Fun,Data,Justificacao
0,1,2023-12-12,Avaria de transporte
1,2,2023-03-27,Doença súbita
2,2,2024-04-25,Procedimentos legais/burocráticos
3,2,2025-04-01,Doença súbita
4,2,2025-09-30,Avaria de transporte
...,...,...,...
230,99,2023-08-09,None
231,99,2025-07-02,Doença súbita
232,99,2023-09-11,None
233,100,2024-06-12,Avaria de transporte


In [150]:
def criar_historico_empresas(funcionarios_df):
    """
    Gera um histórico profissional para os funcionários.
    O primeiro registo (i=0) é SEMPRE o emprego atual na 'bd054' com Data_fim NULL.
    Os registos seguintes (i>0) são empregos anteriores noutras empresas.
    """
    lista_historico = []
    
    
    # Itera sobre cada funcionário
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # 80% de chance de o funcionário ter um histórico (além do atual)
        # Nota: Agora todos terão PELO MENOS 1 registo (o atual)
        num_historicos = random.randint(0, 3) # 0 = só tem o emprego atual

        # Âncora de data. Começa com a data de início do emprego atual
        # A data de início na 'bd054' é algures nos últimos 5 anos
        data_inicio_atual = faker.date_between(start_date='-5y', end_date='today')
        
        # A âncora para empregos anteriores será ANTES desta data
        latest_end_date = data_inicio_atual - timedelta(days=random.randint(30, 365)) # Intervalo entre empregos
            
        for i in range(num_historicos + 1): # +1 para incluir o emprego atual
            
            # --- MUDANÇA CRÍTICA AQUI ---
            if i == 0:
                # O primeiro registo é SEMPRE o emprego atual na bd054
                nome_empresa = 'bd054'
                data_fim = None # Correto, como você disse!
                data_inicio = data_inicio_atual
                # Usamos o cargo real do funcionário na nossa empresa
                cargo = funcionario['Cargo']

            else:
                # Empregos anteriores (i > 0)
                nome_empresa = faker.company()
                data_fim = latest_end_date # Terminou antes do emprego anterior
                data_inicio = data_fim - timedelta(days=random.randint(365, 365 * 5)) # Durou 1-5 anos
                cargo = faker.job()
                
                # Prepara a âncora para o próximo loop (o emprego ainda mais antigo)
                latest_end_date = data_inicio - timedelta(days=random.randint(30, 365))
            
            historico_record = {
                "ID_Fun": id_fun,
                "Nome_empresa": nome_empresa,
                "Cargo": cargo,
                "Data_inicio": data_inicio,
                "Data_fim": data_fim
            }
            lista_historico.append(historico_record)

    return pd.DataFrame(lista_historico)

In [151]:
Historico_empresas = criar_historico_empresas(Funcionarios)
Historico_empresas

,ID_Fun,Nome_empresa,Cargo,Data_inicio,Data_fim
0,1,bd054,Supervisor de Atendimento ao Cliente,2025-06-23,None
1,1,Gonçalves,Terapeuta da fala,2019-12-03,2024-11-25
2,1,Pereira Lda.,Condutor de veículos de tração animal,2016-09-14,2019-01-05
3,1,Soares,Técnico de análises clínicas,2013-10-15,2016-05-13
4,2,bd054,Redator (Copywriter),2023-07-18,None
...,...,...,...,...,...
260,99,Esteves,Auxiliar de enfermagem,2012-03-23,2016-10-14
261,99,Torres S/A,Técnico de emissões de televisão,2008-11-14,2011-07-06
262,100,bd054,Diretor de Operações (COO),2022-03-06,None
263,100,Moreira e Filhos,Técnico de apoio aos utilizadores das TIC,2016-11-28,2021-04-04


In [152]:
def criar_modulo_recrutamento(funcionarios_df, departamentos_df, num_candidatos=50, num_vagas=20):
    """
    Gera dados para as tabelas Candidatos, Vagas, Requisitos_vaga e Candidato_a.
    """
    lista_candidatos = []
    lista_vagas = []
    lista_requisitos = []
    lista_candidaturas = []

    # --- 1. Gerar Candidatos ---
    for i in range(1, num_candidatos + 1):
        candidato_record = {
            "ID_cand": i,
            "Nome": faker.name(),
            "Email": faker.unique.email(),
            "Telemovel": faker.phone_number(),
            "CV": None, # BLOBs são representados como None
            "carta_motivacao": None
        }
        lista_candidatos.append(candidato_record)
    candidatos_df = pd.DataFrame(lista_candidatos)
    ids_candidatos_disponiveis = candidatos_df['ID_cand'].tolist()

    # --- 2. Gerar Vagas ---
    ids_departamentos_disponiveis = departamentos_df['ID_depart'].tolist()
    estados_vaga = ['Aberta', 'Fechada', 'Suspensa']

    for i in range(1, num_vagas + 1):
        vaga_record = {
            "ID_vaga": i,
            "Data_abertura": faker.date_between(start_date='-2y', end_date='today'),
            "Estado": random.choice(estados_vaga),
            "ID_depart": random.choice(ids_departamentos_disponiveis)
        }
        lista_vagas.append(vaga_record)
    vagas_df = pd.DataFrame(lista_vagas)
    
    # --- 3. Gerar Requisitos para cada Vaga ---
    requisitos_possiveis = [
        'Python', 'SQL', 'Gestão de Projetos', 'Comunicação', 'Inglês Fluente',
        'Análise de Dados', 'React', 'Liderança', 'Trabalho em Equipa', 'Excel Avançado',
        'Marketing Digital', 'Contabilidade', 'Java', 'Atendimento ao Cliente'
    ]
    for index, vaga in vagas_df.iterrows():
        # Cada vaga terá entre 2 e 5 requisitos
        num_requisitos = random.randint(2, 5)
        # Usamos random.sample para garantir que os requisitos para a mesma vaga são únicos
        requisitos_para_vaga = random.sample(requisitos_possiveis, num_requisitos)
        
        for req in requisitos_para_vaga:
            requisito_record = {
                "ID_vaga": vaga['ID_vaga'],
                "Requisito": req
            }
            lista_requisitos.append(requisito_record)
    requisitos_df = pd.DataFrame(lista_requisitos)

    # --- 4. Gerar Candidaturas (Candidato_a) ---
    # Para ser mais realista, vamos pegar nos IDs dos funcionários de RH para serem os recrutadores
    id_depart_rh = departamentos_df[departamentos_df['Nome'] == 'Recursos Humanos']['ID_depart'].iloc[0]
    ids_recrutadores = funcionarios_df[funcionarios_df['ID_depart'] == id_depart_rh]['ID_fun'].tolist()
    # Fallback: se não houver ninguém em RH, usamos qualquer funcionário
    if not ids_recrutadores:
        ids_recrutadores = funcionarios_df['ID_fun'].tolist()
        
    estados_candidatura = ['Submetido', 'Em análise', 'Entrevista', 'Rejeitado', 'Rejeitado', 'Rejeitado', 'Contratado']

    # Cada candidato vai candidatar-se a um número aleatório de vagas (0 a 3)
    for id_candidato in ids_candidatos_disponiveis:
        num_candidaturas = random.randint(0, 3)
        if num_candidaturas > 0:
            # Escolhe 'num_candidaturas' vagas aleatórias e únicas para se candidatar
            vagas_aplicadas = vagas_df.sample(num_candidaturas)
            
            for index, vaga in vagas_aplicadas.iterrows():
                # A data da candidatura tem de ser posterior à data de abertura da vaga
                data_candidatura = faker.date_between(start_date=vaga['Data_abertura'], end_date='today')
                
                candidatura_record = {
                    "ID_cand": id_candidato,
                    "ID_Vaga": vaga['ID_vaga'],
                    "Data_cand": data_candidatura,
                    "Estado": random.choice(estados_candidatura),
                    "ID_recrutador": random.choice(ids_recrutadores)
                }
                lista_candidaturas.append(candidatura_record)
    candidato_a_df = pd.DataFrame(lista_candidaturas)
    
    return candidatos_df, vagas_df, requisitos_df, candidato_a_df

In [153]:
Candidatos, Vagas, Requisitos_vaga, Candidato_a = criar_modulo_recrutamento(Funcionarios, Departamentos)

In [154]:
Candidatos

,ID_cand,Nome,Email,Telemovel,CV,carta_motivacao
0,1,Rita de Alves,vitor84@example.com,(351) 965 552 069,None,None
1,2,Guilherme Vicente-Marques,irisnunes@example.org,926 780 297,None,None
2,3,Vítor Torres,diegovaz@example.net,(351) 235781230,None,None
3,4,Violeta Paiva,mendesjuliana@example.net,(351) 931 923 395,None,None
4,5,Márcio Santos,antunesisaac@example.com,(351) 967 444 987,None,None
5,6,Samuel Simões,xavierloureiro@example.net,(351) 922383209,None,None
6,7,João Lourenço,dsousa@example.com,(351) 939653879,None,None
7,8,Xavier Castro,micaelnascimento@example.org,914677432,None,None
8,9,César Paiva,fabiana97@example.net,(351) 915610875,None,None
9,10,Noa Fonseca,crocha@example.com,+351201064676,None,None


In [155]:
Vagas

,ID_vaga,Data_abertura,Estado,ID_depart
0,1,2024-03-23,Fechada,4
1,2,2024-12-07,Suspensa,4
2,3,2024-01-27,Suspensa,2
3,4,2025-01-15,Aberta,2
4,5,2025-01-04,Fechada,6
5,6,2025-07-15,Suspensa,6
6,7,2024-07-17,Aberta,4
7,8,2025-03-19,Suspensa,7
8,9,2025-03-28,Fechada,3
9,10,2024-04-11,Fechada,5


In [156]:
Requisitos_vaga


,ID_vaga,Requisito
0,1,Java
1,1,Excel Avançado
2,1,Python
3,2,Análise de Dados
4,2,Atendimento ao Cliente
...,...,...
65,20,Excel Avançado
66,20,Marketing Digital
67,20,Python
68,20,Liderança


In [157]:
Candidato_a

,ID_cand,ID_Vaga,Data_cand,Estado,ID_recrutador
0,1,19,2025-09-05,Rejeitado,67
1,1,12,2025-07-22,Rejeitado,54
2,2,10,2024-05-14,Em análise,44
3,2,5,2025-10-08,Entrevista,66
4,2,18,2025-02-03,Rejeitado,67
...,...,...,...,...,...
75,47,7,2025-06-26,Entrevista,67
76,48,9,2025-03-29,Entrevista,66
77,49,7,2024-09-07,Rejeitado,89
78,49,16,2025-06-05,Rejeitado,66


In [158]:
def criar_catalogo_formacoes():
    """
    Cria um catálogo fixo e manual de 5 formações com descrições realistas.
    """
    dados_formacoes = [
        {"ID_for": 1, "Nome_formacao": 'Liderança e Gestão de Equipas', "Descricao": 'Desenvolver competências de liderança, motivação e gestão de conflitos para chefes de equipa.', "Data_inicio": date(2023, 10, 5), "Data_fim": date(2023, 10, 10), "Estado": 'Concluida'},
        {"ID_for": 2, "Nome_formacao": 'Cibersegurança para Colaboradores', "Descricao": 'Sensibilizar para as melhores práticas de segurança digital, phishing e proteção de dados.', "Data_inicio": date(2024, 3, 15), "Data_fim": date(2024, 3, 16), "Estado": 'Concluida'},
        {"ID_for": 3, "Nome_formacao": 'Análise de Dados com Python', "Descricao": 'Introdução à análise de dados utilizando as bibliotecas Pandas e Matplotlib.', "Data_inicio": date(2024, 9, 2), "Data_fim": date(2024, 9, 13), "Estado": 'Em curso'},
        {"ID_for": 4, "Nome_formacao": 'Técnicas de Negociação e Vendas', "Descricao": 'Aperfeiçoar técnicas de comunicação, persuasão e fecho de negócios para a equipa comercial.', "Data_inicio": date(2025, 2, 10), "Data_fim": date(2025, 2, 14), "Estado": 'Planeada'},
        {"ID_for": 5, "Nome_formacao": 'Inteligência Emocional no Trabalho', "Descricao": 'Gerir emoções, melhorar o autoconhecimento e a empatia nas relações interpessoais no trabalho.', "Data_inicio": date(2025, 5, 19), "Data_fim": date(2025, 5, 20), "Estado": 'Planeada'}
    ]
    
    return pd.DataFrame(dados_formacoes)

In [159]:
Formacoes = criar_catalogo_formacoes()
Formacoes

,ID_for,Nome_formacao,Descricao,Data_inicio,Data_fim,Estado
0,1,Liderança e Gestão de Equipas,"Desenvolver competências de liderança, motivaç...",2023-10-05,2023-10-10,Concluida
1,2,Cibersegurança para Colaboradores,Sensibilizar para as melhores práticas de segu...,2024-03-15,2024-03-16,Concluida
2,3,Análise de Dados com Python,Introdução à análise de dados utilizando as bi...,2024-09-02,2024-09-13,Em curso
3,4,Técnicas de Negociação e Vendas,"Aperfeiçoar técnicas de comunicação, persuasão...",2025-02-10,2025-02-14,Planeada
4,5,Inteligência Emocional no Trabalho,"Gerir emoções, melhorar o autoconhecimento e a...",2025-05-19,2025-05-20,Planeada


In [160]:
def criar_teve_formacao(funcionarios_df, formacoes_df):
    """
    Gera as inscrições dos funcionários nas formações.
    Garante que as datas de participação do funcionário estão DENTRO
    do período em que a formação esteve ativa.
    """
    lista_teve_formacao = []

    for index, funcionario in funcionarios_df.iterrows():
        if random.random() < 0.6:
            num_formacoes_feitas = random.randint(1, 2)
            formacoes_a_inscrever = formacoes_df.sample(num_formacoes_feitas)
            
            for _, formacao in formacoes_a_inscrever.iterrows():
                
                # --- LÓGICA DE DATA CORRIGIDA AQUI ---
                
                # 1. Definir os limites da formação
                formacao_inicio = formacao['Data_inicio']
                formacao_fim = formacao['Data_fim']
                
                # 2. Gerar uma data de início de participação DENTRO desse período
                # (usamos date_between_dates para trabalhar com objetos date)
                data_inicio_participacao = faker.date_between_dates(date_start=formacao_inicio, date_end=formacao_fim)
                
                # 3. Gerar uma data de fim de participação DENTRO do que resta do período
                data_fim_participacao = faker.date_between_dates(date_start=data_inicio_participacao, date_end=formacao_fim)
                
                # --- FIM DA CORREÇÃO ---

                teve_formacao_record = {
                    "ID_Fun": funcionario['ID_fun'],
                    "ID_for": formacao['ID_for'],
                    "Certificado": None, # BLOB
                    "Data_inicio": data_inicio_participacao, # Data corrigida
                    "Data_fim": data_fim_participacao      # Data corrigida
                }
                lista_teve_formacao.append(teve_formacao_record)

    return pd.DataFrame(lista_teve_formacao)

In [161]:
Teve_Formacao = criar_teve_formacao(Funcionarios, Formacoes)
Teve_Formacao

,ID_Fun,ID_for,Certificado,Data_inicio,Data_fim
0,1,4,None,2025-02-11,2025-02-13
1,1,1,None,2023-10-05,2023-10-08
2,5,2,None,2024-03-15,2024-03-15
3,6,3,None,2024-09-12,2024-09-12
4,6,2,None,2024-03-15,2024-03-15
...,...,...,...,...,...
91,94,3,None,2024-09-09,2024-09-12
92,94,4,None,2025-02-13,2025-02-13
93,95,4,None,2025-02-10,2025-02-12
94,98,1,None,2023-10-05,2023-10-07


In [162]:
FRASES_CRITERIOS = [
    "Avaliação da pontualidade e assiduidade no último semestre.",
    "Desempenho nos projetos de equipa e capacidade de colaboração.",
    "Capacidade de resolução de problemas e tomada de decisão.",
    "Qualidade da comunicação com o cliente e stakeholders internos.",
    "Iniciativa e proatividade na sugestão de novas melhorias.",
    "Qualidade técnica do trabalho entregue e atenção ao detalhe.",
    "Gestão de tempo, cumprimento de prazos e organização."
]

FRASES_AUTOAVALIACAO = [
    "Sinto que atingi todos os meus objetivos principais este semestre.",
    "Completei a formação X e sinto-me mais confiante nas minhas tarefas.",
    "Gostaria de ter mais formação na área de análise de dados para contribuir melhor.",
    "Consegui melhorar a minha organização e gestão de tempo, como foi discutido.",
    "Tive alguma dificuldade em gerir o projeto Y, mas aprendi muito com o processo.",
    "Preciso de focar-me em ser mais conciso nas reuniões de equipa."
]


def criar_avaliacoes(funcionarios_df, num_avaliacoes=40):
    """
    Gera um número fixo de avaliações (ex: 40) com frases realistas,
    IDs de avaliadores inteiros e uma avaliação numérica.

    Notas:
    - Avaliacao_Numerica é um inteiro de 1 a 5 (escala Likert). Pode ser ajustado conforme necessário.
    """
    lista_avaliacoes = []
    
    # Obter uma lista de todos os IDs de funcionários
    ids_funcionarios_disponiveis = funcionarios_df['ID_fun'].tolist()

    # Se não houver funcionários, não podemos fazer nada
    if not ids_funcionarios_disponiveis or len(ids_funcionarios_disponiveis) < 2:
        print("Aviso: É preciso ter pelo menos 2 funcionários para gerar avaliações.")
        return pd.DataFrame()

    for _ in range(num_avaliacoes):
        
        # 1. Escolhe um funcionário para ser AVALIADO
        id_fun_avaliado = random.choice(ids_funcionarios_disponiveis)
        
        # 2. Escolhe um AVALIADOR
        id_avaliador = random.choice(ids_funcionarios_disponiveis)
        
        # 3. Garante que não é a mesma pessoa (autoavaliação é noutro campo)
        while id_avaliador == id_fun_avaliado:
            id_avaliador = random.choice(ids_funcionarios_disponiveis)
            
        # 4. Gera uma data única para a chave primária
        data_avaliacao = faker.date_between(start_date='-3y', end_date='today')

        # 5. 70% de chance de ter uma autoavaliação
        autoavaliacao = random.choice(FRASES_AUTOAVALIACAO) if random.random() < 0.7 else None

        # 6. Avaliação numérica (1 a 5)
        avaliacao_numerica = random.randint(1, 5)
        
        avaliacao_record = {
            "ID_Fun": id_fun_avaliado,
            "ID_avaliador": id_avaliador, # Isto será sempre um INT
            "Data": data_avaliacao,
            "Avaliacao": None, # BLOB
            "Criterios": random.choice(FRASES_CRITERIOS), # Frase realista
            "Autoavaliacao": autoavaliacao, # Frase realista ou None
            "Avaliacao_Numerica": avaliacao_numerica
        }
        
        lista_avaliacoes.append(avaliacao_record)

    # Criar o DataFrame
    avaliacoes_df = pd.DataFrame(lista_avaliacoes)

    # Salvaguarda: Remove duplicados na chave primária (caso tenhamos gerado o mesmo trio por acaso)
    avaliacoes_df = avaliacoes_df.drop_duplicates(subset=['ID_Fun', 'ID_avaliador', 'Data'], keep='first')
    return avaliacoes_df

In [163]:
Avaliacoes = criar_avaliacoes(Funcionarios)
Avaliacoes

,ID_Fun,ID_avaliador,Data,Avaliacao,Criterios,Autoavaliacao,Avaliacao_Numerica
0,5,46,2025-08-05,None,Qualidade da comunicação com o cliente e stake...,"Tive alguma dificuldade em gerir o projeto Y, ...",4
1,14,15,2025-04-12,None,Avaliação da pontualidade e assiduidade no últ...,None,2
2,80,8,2025-01-16,None,Desempenho nos projetos de equipa e capacidade...,None,1
3,63,27,2024-12-06,None,Desempenho nos projetos de equipa e capacidade...,Sinto que atingi todos os meus objetivos princ...,4
4,46,30,2023-12-12,None,Iniciativa e proatividade na sugestão de novas...,None,1
5,95,65,2025-08-01,None,"Gestão de tempo, cumprimento de prazos e organ...","Tive alguma dificuldade em gerir o projeto Y, ...",3
6,28,18,2023-09-10,None,Desempenho nos projetos de equipa e capacidade...,None,3
7,80,68,2025-09-02,None,Iniciativa e proatividade na sugestão de novas...,Sinto que atingi todos os meus objetivos princ...,2
8,48,32,2024-08-23,None,Qualidade da comunicação com o cliente e stake...,Sinto que atingi todos os meus objetivos princ...,1
9,47,50,2023-07-26,None,"Gestão de tempo, cumprimento de prazos e organ...",Consegui melhorar a minha organização e gestão...,5


In [164]:
def criar_utilizadores_permissoes(funcionarios_df):
    """
    Gera dados para as tabelas Utilizadores e Permissoes.
    Cria um utilizador para cada funcionário e atribui permissões aleatórias.
    """
    lista_utilizadores = []
    lista_permissoes = []
    
    # A sua lista de permissões válidas
    permissoes_possiveis = ['Funcionario', 'Manager', 'RH', 'Admin', 'Financeiro']

    # Itera sobre CADA funcionário para criar uma conta de utilizador
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # --- 1. Criar o registo na tabela Utilizadores ---
        utilizador_record = {
            "ID_Fun": id_fun,
            # Gera uma password aleatória. 'special_chars=False' para caber no VARCHAR(30)
            "Password": faker.password(length=12, special_chars=False) 
        }
        lista_utilizadores.append(utilizador_record)
        
        # --- 2. Criar os registos na tabela Permissoes ---
        
        # Cada utilizador terá entre 1 e 3 permissões
        num_permissoes = random.randint(1, 3)
        
        # Escolhe 'num_permissoes' permissões únicas da lista
        permissoes_para_este_user = random.sample(permissoes_possiveis, num_permissoes)
        
        # Cria um registo para cada permissão atribuída
        for perm in permissoes_para_este_user:
            permissao_record = {
                "ID_Fun": id_fun,
                "Permissao": perm
            }
            lista_permissoes.append(permissao_record)

    utilizadores_df = pd.DataFrame(lista_utilizadores)
    permissoes_df = pd.DataFrame(lista_permissoes)
    
    return utilizadores_df, permissoes_df

In [165]:
Utilizadores, Permissoes = criar_utilizadores_permissoes(Funcionarios)

In [166]:
Utilizadores

,ID_Fun,Password
0,1,NBESpl5Ky4RJ
1,2,4zMCEuarEtv4
2,3,H9jDvKOp832h
3,4,7tHFYkgke6o8
4,5,8jVNqgTi2ldg
...,...,...
95,96,e2gAuywnYz9Q
96,97,1W8ufGgh8KQw
97,98,yhDZaHrE2ntj
98,99,g14aDtzRocDw


In [167]:
Permissoes

,ID_Fun,Permissao
0,1,Manager
1,1,RH
2,1,Funcionario
3,2,Admin
4,3,Manager
...,...,...
196,99,Manager
197,99,Funcionario
198,100,Admin
199,100,Funcionario


In [168]:
import pandas as pd
from datetime import date, datetime
import math

print("A iniciar a geração do ficheiro 'dados_insercao.sql' (um INSERT por tabela)...")

# --- Função Auxiliar (sem alterações) ---
def formatar_valor_sql(valor):
    if pd.isna(valor) or valor is None:
        return "NULL"
    if isinstance(valor, str):
        return "'" + valor.replace("'", "''") + "'"
    if isinstance(valor, (int, float)):
        return str(valor)
    if isinstance(valor, (date, datetime)):
        return f"'{valor.isoformat()}'"
    return f"'{str(valor)}'"

# --- Dicionário com todos os DataFrames (sem alterações) ---
todos_dataframes = {
    "Remuneracoes": Remuneracoes,
    "Salario": Salarios,
    "Beneficios": Beneficios,
    "Ferias": Ferias,
    "Dependentes": Dependentes,
    "Faltas": Faltas,
    "Historico_empresas": Historico_empresas,
    "Candidatos": Candidatos,
    "Vagas": Vagas,
    "Candidato_a": Candidato_a,
    "Requisitos_vaga": Requisitos_vaga,
    "Formacoes": Formacoes,
    "Teve_Formacao": Teve_Formacao,
    "Avaliacoes": Avaliacoes,
    "Utilizadores": Utilizadores,
    "Permissoes": Permissoes
}

# --- NOVO: Função para gerar UM ÚNICO INSERT por tabela ---
def escrever_insert_unico_sql(f, df, nome_tabela, colunas_sql=None):
    """
    Escreve um DataFrame num ficheiro SQL usando UMA ÚNICA instrução INSERT.
    """
    if df.empty:
        f.write(f"\n-- Tabela {nome_tabela} está vazia. A saltar.\n")
        return

    f.write(f"\n-- Dados para {nome_tabela}\n")
    
    if colunas_sql:
        cols_sql = colunas_sql
    else:
        cols_sql = ", ".join(df.columns)
    
    # Escreve o início da instrução
    f.write(f"INSERT INTO {nome_tabela} ({cols_sql}) VALUES\n")
    
    linhas_de_valores = []
    # Itera por TODAS as linhas do DataFrame
    for row in df.itertuples(index=False, name=None):
        valores_formatados = f"({', '.join([formatar_valor_sql(val) for val in row])})"
        linhas_de_valores.append(valores_formatados)
    
    # Junta todas as linhas de valores com uma vírgula
    f.write(",\n".join(linhas_de_valores))
    f.write(";\n") # Termina a instrução INSERT

# --- Processo de Escrita do Ficheiro (Atualizado) ---
try:
    with open('dados_insercao.sql', 'w', encoding='utf-8') as f:
        f.write("-- Script de Inserção de Dados Gerado Automaticamente\n")

        # 1. Departamentos (Inserir com gerente NULL)
        df_deptos_temp = Departamentos[['ID_depart', 'Nome']].copy()
        df_deptos_temp['ID_gerente'] = None
        escrever_insert_unico_sql(f, df_deptos_temp, "Departamentos", colunas_sql="ID_depart, Nome, ID_gerente")
        
        # 2. Funcionarios
        escrever_insert_unico_sql(f, Funcionarios, "Funcionarios")

        # 3. UPDATE Departamentos (Agora com os gerentes)
        f.write("\n-- 3. Atualização dos Gerentes nos Departamentos\n")
        for row in Departamentos.itertuples(index=False):
            if pd.notna(row.ID_gerente):
                f.write(f"UPDATE Departamentos SET ID_gerente = {int(row.ID_gerente)} WHERE ID_depart = {row.ID_depart};\n")

        # 4. Inserção de todas as outras tabelas
        f.write("\n-- 4. Inserção das restantes tabelas\n")
        for nome_tabela, df in todos_dataframes.items():
            
            # Lógica especial para Candidato_a para renomear colunas
            if nome_tabela == 'Candidato_a':
                escrever_insert_unico_sql(f, df, nome_tabela, colunas_sql="ID_cand, ID_Vaga, Data_cand, Estado, ID_recrutador")
            else:
                escrever_insert_unico_sql(f, df, nome_tabela)


    print("\n--- SUCESSO! ---")
    print("O ficheiro 'dados_insercao.sql' foi gerado com sucesso.")
    print("Cada tabela tem agora UM ÚNICO comando INSERT com todos os seus dados.")

except Exception as e:
    print(f"\n--- ERRO ---")
    print(f"Ocorreu um erro ao gerar o ficheiro: {e}")
    print("Verifique os nomes dos seus DataFrames.")

A iniciar a geração do ficheiro 'dados_insercao.sql' (um INSERT por tabela)...

--- SUCESSO! ---
O ficheiro 'dados_insercao.sql' foi gerado com sucesso.
Cada tabela tem agora UM ÚNICO comando INSERT com todos os seus dados.
